## Aquecendo


In [0]:
SELECT current_catalog(), current_schema()


In [0]:
USE CATALOG smart_claims_dev

In [0]:
SELECT current_catalog(), current_schema()


In [0]:
USE SCHEMA 00_landing

In [0]:
SELECT current_catalog(), current_schema()


## EXPLORE THE DATA SOURCE FILES



In [0]:
LIST '/Volumes/smart_claims_dev/00_landing/sql_server'

In [0]:
%python
files = dbutils.fs.ls(
    '/Volumes/smart_claims_dev/00_landing/sql_server'
)
display(files)

Query os arquivos CSV usando o path do volume

In [0]:
SELECT *
FROM csv.`/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv`
--drop the table if it exists for demonstration

## Create a STREAMING TABLE using Databricks Auto Loader

In [ ]:
DROP TABLE IF EXISTS smart_claims_dev.01_bronze.claims;

In [0]:
CREATE OR REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims
SCHEDULE EVERY 1 WEEK
AS
SELECT *
FROM read_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv',
  header => true
)



In [ ]:
SELECT * FROM smart_claims_dev.01_bronze.claims
LIMIT 10;

In [ ]:
DESCRIBE TABLE EXTENDED smart_claims_dev.01_bronze.claims;

In [ ]:
DESCRIBE HISTORY smart_claims_dev.01_bronze.claims;

In [ ]:
REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims;

In [ ]:
SELECT * FROM smart_claims_dev.01_bronze.claims

In [ ]:
DESCRIBE HISTORY smart_claims_dev.01_bronze.claims;